# Import modules

In [38]:
import pandas as pd
import numpy as np
import requests

import sklearn
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/c1977808/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# Read data

In [4]:
def read_data(path_pos, path_neg):
    pos = pd.read_csv(path_pos, sep="\n", header=None, names=['review'])
    pos['positive']=1
    neg = pd.read_csv(path_neg, sep="\n", header=None, names=['review'])
    neg['positive']=0
    combined_df = pos.append(neg)
    combined_df = shuffle(combined_df, random_state=42)
    return(combined_df)

In [5]:
# read in training data
train = read_data(path_pos="Data/IMDb/train/imdb_train_pos.txt",
                  path_neg="Data/IMDb/train/imdb_train_neg.txt")

dev = read_data(path_pos="Data/IMDb/dev/imdb_dev_pos.txt",
                path_neg="Data/IMDb/dev/imdb_dev_neg.txt")

test = read_data(path_pos="Data/IMDb/test/imdb_test_pos.txt",
                 path_neg="Data/IMDb/test/imdb_test_neg.txt")

# Explore data and shuffle

In [6]:
train.head()

,review,positive
4016,This is a comedy/romance movie directed by And...,0
6475,"During the Sci-Fi TZ marathon of January 31, 1...",1
5684,radio is possibly one of the best films i have...,0
862,I was -Unlike most of the reviewers- not born ...,1
5970,"When i started watching ""Surface""for the first...",1


In [7]:
print("No of positive reviews\n-----")
print(train['positive'].value_counts()[1])
print("\nNo of negative reviews\n-----")
print(train['positive'].value_counts()[0])

No of positive reviews
-----
7483

No of negative reviews
-----
7517


# Feature 1 - tf-idf

The first feature is based on a vocabulary of tokens collected from the training data. Term Frequency times Inverse Document Frequency will be used. Term Frequency (tf) counts the number of times a token in the vocabulary is used in each review, relative to how frequently it appears in that review. This acts as a way of normalising the count. Inverse Document Frequency (idf) penalises tokens that appear across many reviews, as these terms offer less information than those that appear in fewer reviews. 

## Create set of stopwords to remove later

In [8]:
# take set of stopwords from nltk
stopwords=set(nltk.corpus.stopwords.words('english'))
# manually add more punctuation
stopwords.add(".")
stopwords.add(",")
stopwords.add("--")
stopwords.add("``")
stopwords.add("#")
stopwords.add("@")
stopwords.add(":")
stopwords.add("'s")
stopwords.add("’")
stopwords.add("...")
stopwords.add("n't")
stopwords.add("'re")
stopwords.add("'")
stopwords.add("-")
stopwords.add(";")
stopwords.add("/")
stopwords.add(">")
stopwords.add("<")
stopwords.add("br")
stopwords.add("(")
stopwords.add(")")
stopwords.add("''")
stopwords.add("&")

## Define custom transformers

Need to create a simple transormer so just the dataset can be used to feed into the full pipeline later. Currently, feature 1 pipeline takes `train['review']` while feature 2 pipeline takes `train` as it's argument. This needs to be consistent.

In [9]:
class selectReview(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return(X['review'])

## Create transformation pipeline

In [10]:
feature_1_vocab = Pipeline([
    ('select_review', selectReview()),
    ('count', CountVectorizer(stop_words=stopwords, max_features=500)),
    ('tfidf', TfidfTransformer())
])

# Feature 2 - tf-idf (bi-grams)

In [11]:
feature_2_vocab = Pipeline([
    ('select_review', selectReview()),
    ('count', CountVectorizer(stop_words=stopwords, max_features=500, ngram_range=(2,2))),
    ('tfidf', TfidfTransformer())
])

# Feature 3 - Sentiment

In [12]:
vader = SentimentIntensityAnalyzer()

## Define custom transformers

In [13]:
class getSentiment(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        features_array=[]
        for index, row in X.iterrows():
            pos = vader.polarity_scores(row['review'])['pos']
            neu = vader.polarity_scores(row['review'])['neu']
            neg = vader.polarity_scores(row['review'])['neg']
            features_array.append([pos, neu, neg])
        return(np.asarray(features_array))

## Create transformation pipeline

In [14]:
feature_3_sentiment = Pipeline([
    ('get_sentiment', getSentiment())
])

# Combine all features

In [15]:
feature_engineering = FeatureUnion(transformer_list=[
    ("feature_1_vocab", feature_1_vocab),
    ("feature_2_vocab", feature_2_vocab),
    ("feature_3_sentiment", feature_3_sentiment)
])

In [16]:
X_train = feature_engineering.fit_transform(train)

In [17]:
print(X_train.shape)

(15000, 1003)


# Run and evaluate different models

In [20]:
y_train = np.asarray(train['positive'])
y_dev = np.asarray(dev['positive'])
X_dev = feature_engineering.transform(dev)

## SVM

### Linear

In [21]:
svm_clf = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("svm_clf", sklearn.svm.LinearSVC(loss='hinge'))
])

In [22]:
svm_clf.fit(X_train, y_train)
svm_clf_pred = svm_clf.predict(X_dev)

/home/c1977808/.local/share/virtualenvs/c1977808-tkY5RK3O/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [23]:
print(classification_report(y_dev, svm_clf_pred))

              precision    recall  f1-score   support

           0       0.87      0.78      0.82      2482
           1       0.80      0.89      0.84      2518

    accuracy                           0.83      5000
   macro avg       0.84      0.83      0.83      5000
weighted avg       0.84      0.83      0.83      5000



### RBF

In [103]:
rbf_svm_clf = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("svm_clf", sklearn.svm.SVC(kernel="rbf"))
])

In [104]:
rbf_svm_clf.fit(X_train, y_train)
rbf_svm_clf_pred = rbf_svm_clf.predict(X_dev)

In [26]:
print(classification_report(y_dev, rbf_svm_clf_pred))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2482
           1       0.85      0.85      0.85      2518

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



### Polynomial

In [27]:
poly_svm_clf = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("svm_clf", sklearn.svm.SVC(kernel="poly", degree=3, coef0=1))
])

In [28]:
poly_svm_clf.fit(X_train, y_train)
poly_svm_clf_pred = poly_svm_clf.predict(X_dev)

In [29]:
print(classification_report(y_dev, poly_svm_clf_pred))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2482
           1       0.85      0.85      0.85      2518

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



## Decision Tree

In [36]:
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf.fit(X_train, y_train)
tree_pred = tree_clf.predict(X_dev)

In [37]:
print(classification_report(y_dev, tree_pred))

              precision    recall  f1-score   support

           0       0.68      0.80      0.74      2482
           1       0.77      0.64      0.69      2518

    accuracy                           0.72      5000
   macro avg       0.72      0.72      0.72      5000
weighted avg       0.73      0.72      0.72      5000



# Fine tune hyperparameters

Use `GridSearchCV` on shortlisted models to find the best combination of hyperparameters.

In [122]:
param_grid = {
    'svm_clf__C': [0.1, 1.0],
}

In [85]:
grid_search = GridSearchCV(svm_clf, param_grid, scoring='f1_weighted')

In [ ]:
grid_search.fit(X_train, y_train)

# Run feature selection

In [121]:
for i in [10,100,500,1000]:
    selector = SelectKBest(chi2, k=i)
    X_train_reduced = selector.fit_transform(X_train, np.asarray(train['positive']))
    X_dev_reduced = selector.transform(X_dev)
    rbf_svm_clf.fit(X_train_reduced, y_train)
    rbf_svm_clf_pred = rbf_svm_clf.predict(X_dev_reduced)
    f_measure = f1_score(y_dev, rbf_svm_clf_pred, average='weighted')
    print('\nNo of features: '+str(i))
    print('F-measure: '+str(round(f_measure,3)))


No of features: 10
F-measure: 0.738

No of features: 100
F-measure: 0.815

No of features: 500
F-measure: 0.844

No of features: 1000
F-measure: 0.853
